Maximum Likelihood Estimation VS Maximum A Posteriori Estimation
=========================================================

In this notebook we will study empirically the properties of Maximum Likelihood Estimation (MLE) and Maximum A Posteriori (MAP) estimation. The notebook is mostly self-contained, but I will assume that you're familiar with probability mass functions, probability density functions, and the notation in Martha's notes.

Suppose that we want to etimate the rate parameter $\lambda$ of a Poisson distribution by sammpling an independent sample $\mathcal{D}$ of size $n$. 
We will do this by finding the most likely model from a *hypothesis space* $\mathcal{F}$ given the data. 
In other words, given the examples $\{x_i\}^n_{i=1}$ in the data set $\mathcal{D}$, we want to find the model $f$ in the hypothesis class $\mathcal{F}$ that maximizes $p(f|\mathcal{D})$ &mdash; the posterior probability. 
Using Bayes rule, we can split up this probability into three terms:

\begin{align}
p(f|\mathcal{D}) = \frac{p(f \cap \mathcal{D})}{ p(\mathcal{D})} = \frac{p(\mathcal{D} | f) p(f)}{p(\mathcal{D})}
\end{align}

Note that $p(\mathcal{D})$, the probability of observing the data without any prior knowledge of $f$, does not have any effect in the maximization problem since it doesn't depend on $f$. Hence, we will omit it and will consider instead:

\begin{align}
p(f|\mathcal{D}) \propto p(\mathcal{D} | f) p(f),
\end{align}

where $\propto$ means proportional to. The equation above has two main terms: $p(\mathcal{D} | f)$, the probability of the data given a model, and $p(f)$ the probability of the model. If we consider only $p(\mathcal{D} | f)$, then we are interested in the model $f$ that is most likely to produce the data &mdash; this is the case of MLE. If we consider both terms together, then we are interested in the model $f$ that is most likely to produce the data, but that is also most congruent with our believe about the probability distribution of $f$ &mdash; this is the case of MAP estimation. 

You can think about the hypothesis space as specifying the type of models that we will consider for our solution.
For example, you could consider the set of all linear functions, the set of all neural networks of certain size and architecture, or the set of all normal distributions.
In this way, the hypothesis class allows you to set some constrains on the possible solutions. 
In our problem, our hypothesis class $\mathcal{F}$ will be all the possible Poisson distributions, which are fully characterized by the rate parameter $\lambda$. 
Hence, instead of writing $p( \mathcal{D} | f)$, we will write $p(\mathcal{D}|\lambda)$ &mdash; the probability of observing the data set $\mathcal{D}$ given that observations are distributed according to a Poisson distribution with rate parameter $\lambda$.

For the rest of the notebok, we will assume that, unbeknownst to us, the true value of $\lambda$ is 2.


## Maximum Likelihood Estimation

First, let's find an estimate $\hat{\lambda}$ using Maximum Likelihood Estimation (MLE). Knowing that the each element $\{x_i\}^{n}_{i=1}$ is distributed according to a Poisson distribution implies that the probability of observing the data given $\lambda$ is:

\begin{align}
\label{eq:likelihood}
p(\mathcal{D} | \lambda) &= p( x_1, x_2, ..., x_n | \lambda )\tag{1} \\
                         &= \prod^{n}_{i=1} p(x_i | \lambda).\tag{2}
\end{align}

Note that the step from line 1 to line 2 is possible because the samples are independent. The next step is to find the value, $\hat{\lambda}$ that maximizes the likelihood $p( \mathcal{D} | \lambda)$. To do so, we can take the derivative with respect to $\lambda$ in Equation (\ref{eq:likelihood}). However, it will often be easier to derivate the log-likelihood function instead, which has the same maximum because of the logarithmic function is monotonically increasing. Hence, instead of taking the derivative of Equation (\ref{eq:likelihood}), we can take the derivative of:

\begin{align}
    \ln p(\mathcal{D} | \lambda) &= \sum^{n}_{i=1} \ln p(x_i | \lambda)
                                 = \sum^{n}_{i=1} \ln \frac{\lambda^{x_i} e^{-\lambda}}{x_i!} 
                                 = \sum^{n}_{i=1} x_i \ln \lambda -  \lambda - \ln x_i!
                                 = \ln \lambda \sum^n_{i=1} x_i -n \lambda - \sum^n_{i=1} \sum^{x_i}_{j=1} \ln j.
\end{align}

Note that we used the fact that the probability mass function of $x_i$ is $p(x_i | \lambda) = \frac{\lambda^{x_i} e^{-\lambda}}{x_i!}$. Now by taking the derivative with respect to $\lambda$, setting it equal to zero, and solving for $\lambda$ we get:

\begin{align}
& \frac{d}{d \lambda} \ln p(\mathcal{D} | \lambda) = \frac{\sum^n_{i=1} x_i}{ \lambda } - n = 0 \\
& \Rightarrow \hat{\lambda} = \frac{\sum^n_{i=1} x_i }{n}.
\end{align}

Now that we have derived the MLE estimate of $\lambda$, let's see how accurate it is based on differen sample sizes. For this small experiment, we will use sample sizes $n$ in \{5, 10, 20, 40, 80, 160, 320, 640, 1280\}.

Let's start by importing the necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
from scipy.stats import t

Now let's define a Poisson probability distribution with rate parameter $\lambda$ equal to 2 and draw a sample sample of 10.

In [ ]:
true_lambda = 2
dist = poisson(true_lambda)
print(dist.rvs(10))

Note that even if the expected value of the distribution is 2, most of the values in the sample are not equal to 2. It is possible that you might get very lucky and have all values being equal to 2. In such case, rerun the code and see if it happens again (the probability of it happening even once is approximately 0.000002).

Now, let's see how the MLE estimate varies according to the sample size.

In [ ]:
def compute_mle_estimate(sample, n):
    return np.sum(random_sample) / n

sample_sizes = [5, 10, 20, 40, 80, 160, 320, 640, 1280]

for n in sample_sizes:
    # draw a random sample of size n
    random_sample = dist.rvs(n)
    # compute the mle estimate
    mle_estimate = compute_mle_estimate(random_sample, n)
    # print results
    print("=== sample size =", n, "===")
    print("\t MLE estimate is:", np.round(mle_estimate,4))

Run the previous code several times and see how the results change every time. Generally, you will see that the estimate varies less as the sample size increases. However, as you may note, it is difficult to understand the variability of the estimate based on this. To gain more insight, let's compute several estimates using the same sample size and  compute the average and standard deviation of the estimates.

In [ ]:
# Box plot of the spread estimate for each sample size
def boxplot_of_estimates(data, mle=True):
    plt.boxplot(data)
    loc, labels = plt.xticks()
    plt.xticks(loc, sample_sizes)
    plt.xlabel("Sample Size")
    method_name = "MLE" if mle else "MAP"
    plt.ylabel("Average of Value of the " + method_name + " Estimate")
    plt.ylim([0,5])
    plt.show()
    
number_of_samples = 1000 # number of estimates computed for each sample size

raw_data = np.zeros((number_of_samples, len(sample_sizes)), dtype=np.float64)
mle_estimates_avg = np.zeros(len(sample_sizes), dtype=np.float64)
mle_estimates_stddev = np.zeros(len(sample_sizes), dtype=np.float64)

for j, n in enumerate(sample_sizes):
    temp_averages = np.zeros(number_of_samples, dtype=np.float64)
    
    for i in range(number_of_samples):
        # draw a random sample of size n
        random_sample = dist.rvs(n)
        # compute the mle estimate
        mle_estimate = np.sum(random_sample) / n
        temp_averages[i] = mle_estimate
        raw_data[i, j] = mle_estimate
        
    # computing the average of the mle estimates
    mle_estimates_avg[j] = np.average(temp_averages)
    mle_estimates_stddev[j] = np.std(temp_averages, ddof=1)
    print("=== sample size =", n, "===")
    print("\taverage of the MLE estimate:", np.round(mle_estimates_avg[j], 4))
    print("\tstandard deviation of the MLE estimate:", np.round(mle_estimates_stddev[j], 4))

boxplot_of_estimates(raw_data, True)
        

Note that the average of the estimates, regardless of the sample size, is close to the true value. 
This indicates that our estimates are unbiased (it is worth noting that this is not always guaranteed for the MLE estimate). 
However, note that the standard deviation of our estimates decreases as the sample size increases. 
This means that even if the MLE estimate computed using a sample size of 5 is unbiased, the variability of the estimate can result in an estimate that is far away from the true value.

## Maximum A Posteriori 

Now, let's suppose that we have some prior information about the distribution the rate parameter $\lambda$ and use such information for computing our estimate. Assume that we believe that our paramter is distributed to according to $p(\lambda)$. In order to add prior information to our parameter estimation, we can use Maximum A Posteriori (MAP) estimation where we are interested in finding the parameter $\lambda$ that maximizes $p(\mathcal{D}|\lambda) p(\lambda)$.

Once again, we can find the value of $\lambda$ that maximizes the likelihood by taking the derivative with respect to $\lambda$ and setting it to zero. And once again, we can take the derivative of the log-likelihood function instead.

\begin{align}
\ln p( \mathcal{D} | \lambda) p(\lambda) = \ln p( \mathcal{D} | \lambda) + \ln p(\lambda).
\end{align}

We know from the previous section that:

\begin{align}
\frac{d}{d\lambda} p(\mathcal{D} | \lambda) = \frac{\sum^n_{i=1} x_i}{ \lambda } - n.
\end{align}

Hence, we just need to find the derivative of $p(\lambda)$. The derivative will depend on the particular distribution that we use for our prior believe. For example, if we believe that the parameter $\lambda$ follows a Uniform distribution with lower bound $a$ and upper bound $b$, then $p(\lambda) = \frac{1}{b-a}$. However, note that in that case, the derivative of $p(\lambda)$ with respect to $\lambda$ is equal to zero. Hence, when using a uniform prior, the MAP estimate is going to be the same as the MLE estimate. 

Now let's asusme that we believe that our parameter $\lambda$ is distributed according to a normal distribution with unit variance and some mean $\mu$. In such case, we have:

\begin{align}
& \ln p(\lambda) = \ln \frac{1}{\sqrt{2 \pi \sigma^2}} e^{-\frac{(\lambda - \mu)^2}{2 \sigma^2}} 
                = - \ln \sqrt{2 \pi \sigma^2} - \frac{(\lambda - \mu)^2}{2 \sigma^2},   \\
& \frac{d}{d\lambda} \ln p(\lambda) = \frac{\mu - \lambda}{\sigma^2}.
\end{align}

Then, by putting all together we get:

\begin{align}
& \ln p( \mathcal{D} | \lambda) p(\lambda) = 
    \frac{\sum^n_{i=1} x_i}{ \lambda } - n + \frac{\mu - \lambda}{\sigma^2}. = 0, \\
& \Rightarrow \sigma^2 \sum^n_{i=1} x_i = \lambda^2 + \lambda (\sigma^2 n - \mu) \\
& \Rightarrow \sigma^2 \sum^n_{i=1} x_i + \frac{1}{4}(\sigma^2 n-\mu)^2 = 
    \lambda^2 + \lambda (\sigma^2 n - \mu) + \frac{1}{4}(\sigma^2 n-\mu)^2 \\
& \Rightarrow \sigma^2 \sum^n_{i=1} x_i + \frac{1}{4}(\sigma^2 n-\mu)^2 
    = (\lambda + \frac{1}{2}(\sigma^2 n-\mu))^2 \\
& \Rightarrow \sqrt{\sigma^2 \sum^n_{i=1} x_i + \frac{1}{4}(\sigma^2 n-\mu)^2} = 
    \lambda + \frac{1}{2}(\sigma^2 n-\mu) \\
& \Rightarrow \hat{\lambda} =
    \sqrt{\sigma^2 \sum^n_{i=1} x_i + \frac{1}{4}(\sigma^2 n-\mu)^2} - \frac{1}{2}(\sigma^2 n-\mu).
\end{align}

Now, let's repeat the previous experiment and see how the values of $\mu$ and $\sigma$ affect our estimate. First let's study the way $\sigma$ affects our estimate. For this purpose, let's use a value of $\mu$ equal to two.

In [ ]:
def compute_map_estimate_with_normal_prior(data, n, mu, sigma):
    return np.sqrt(sigma**2 * np.sum(data)  + (1/4) * np.power((sigma**2 * n - mu), 2)) - (1/2) * (sigma**2 * n - mu)

mu = 2
sigmas = [0.1, 0.2, 0.5, 1, 2, 5, 10, 100]


for sigma in sigmas:
    raw_data = np.zeros((number_of_samples, len(sample_sizes)), dtype=np.float64)
    map_estimates_avg = np.zeros(len(sample_sizes), dtype=np.float64)
    map_estimates_stddev = np.zeros(len(sample_sizes), dtype=np.float64)
    
    print("===== sigma =", sigma, "=====")
    for j, n in enumerate(sample_sizes):
        temp_averages = np.zeros(number_of_samples, dtype=np.float64)
        for i in range(number_of_samples):
            # draw a random sample of size n
            random_sample = dist.rvs(n)
            # compute the mle estimate
            map_estimate = compute_map_estimate_with_normal_prior(random_sample, n, mu, sigma)
            temp_averages[i] = map_estimate
            raw_data[i, j] = map_estimate
        # computing the average of the mle estimates
        map_estimates_avg[j] = np.average(temp_averages)
        map_estimates_stddev[j] = np.std(temp_averages, ddof=1)
        print("\t=== sample size =", n, "===")
        print("\t\taverage of the MAP estimate:", np.round(map_estimates_avg[j],4), 
              "standard deviation of the MAP estimate:", np.round(map_estimates_stddev[j],4))
        
    # Box plot of the spread of the MAP estimate for each sample size
    boxplot_of_estimates(raw_data, False)
        

There are three main insights we can gain from the experiment above. First, we see the same pattern as in the case of MLE: the larger the sample size, the smaller the variance. Second, the value of $\sigma$ can considerably decrease the variance of our estimate. Third, even with very large values of $\sigma$, the variance of the MAP estimate is not considerably larger than the variance of the MLE estimate. This is the advantage of the MAP estimate; by using prior information when computing $\hat{\lambda}$, we can reduce the variance of our estimate

Now, let's study how the value of the parameter $\mu$ affects our estimate. For this part, we will use a value of $\sigma$ of 0.5.

In [ ]:
mus = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
sigma = 0.5

for mu in mus:
    raw_data = np.zeros((number_of_samples, len(sample_sizes)), dtype=np.float64)
    map_estimates_avg = np.zeros(len(sample_sizes), dtype=np.float64)
    map_estimates_stddev = np.zeros(len(sample_sizes), dtype=np.float64)
    
    print("===== mu =", mu, "=====")
    for j, n in enumerate(sample_sizes):
        temp_averages = np.zeros(number_of_samples, dtype=np.float64)
        
        for i in range(number_of_samples):
            # draw a random sample of size n
            random_sample = dist.rvs(n)
            # compute the map estimate
            map_estimate = compute_map_estimate_with_normal_prior(random_sample, n, mu, sigma)
            temp_averages[i] = map_estimate
            raw_data[i, j] = map_estimate
            
        # computing the average of the map estimates
        map_estimates_avg[j] = np.average(temp_averages)
        map_estimates_stddev[j] = np.std(temp_averages, ddof=1)
        print("\t=== sample size =", n, "===")
        print("\t\taverage of the MAP estimate:", np.round(map_estimates_avg[j],4), 
              "standard deviation of the MAP estimate:", np.round(map_estimates_stddev[j],4))
        

    # Box plot of the spread of the MAP estimate for each sample size
    boxplot_of_estimates(raw_data, False)

This clearly illustrates the disadvantage of MAP estimation. If our prior is very wrong, we can severely bias our estimate. Nevertheless, as our sample size increases, the initial bias of our prior has less influence on our estimate.

It is important to note that there is no "best solution" when considering whether to use MLE or MAP for estimating parameters. Each of these methods represent different mathematical tools that you can use for your application. The "best solution" needs to be tailored to the particular problem you are trying to solve. 